In [175]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

In [176]:
gs = pd.read_csv('Data/gender_submission.csv')
test = pd.read_csv('Data/test.csv')
train = pd.read_csv('Data/train.csv')

In [177]:
mean_Age = (train['Age'].mean() + test['Age'].mean())/2

In [178]:
Surv_train = train['Survived']
train.drop(['Survived', 'Name','Ticket','Cabin','Embarked','PassengerId'], axis='columns', inplace=True)
test.drop(['Name','Ticket','Cabin','Embarked','PassengerId'], axis='columns', inplace=True)

In [179]:
test = test.fillna(mean_Age)

In [180]:
train = train.fillna(mean_Age)

In [181]:
train.loc[train["Sex"] == "male","Sex"] = 0
train.loc[train["Sex"] == "female","Sex"] = 1
test.loc[test["Sex"] == "male","Sex"] = 0
test.loc[test["Sex"] == "female","Sex"] = 1

In [182]:
del gs['PassengerId']

In [183]:
decision = ExtraTreesClassifier(n_estimators=800)

In [184]:
decision.fit(train, Surv_train)
pred = decision.predict(test)

In [185]:
print('Accuracy: \n', accuracy_score(gs, pred, normalize = True))

Accuracy: 
 0.8086124401913876
